In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.cuda.is_available()

True

In [2]:
from src.qkv_v0.interface import qkvkernel

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt21cu118/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt21cu118/lib


Using /home/max/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Creating extension directory /home/max/.cache/torch_extensions/py311_cu118/qkv_v0...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu118/qkv_v0/build.ninja...
Building extension module qkv_v0...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] /home/max/miniconda3/envs/xlstmpt21cu118/bin/x86_64-conda-linux-gnu-c++ -MMD -MF interface.o.d -DTORCH_EXTENSION_NAME=qkv_v0 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC -isystem /home/max/miniconda3/envs/xlstmpt21cu118/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /home/max/myrepos/vlstm_cuda/src/qkv_v0/interface.cc -o interface.o 
[2/3] /home/max/miniconda3/envs/xlstmpt21cu118/bin/nvcc  -ccbin /home/max/miniconda3/env

Loading extension module qkv_v0...


### qkvtest

In [3]:
S = 8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 4 # dim per head
DTYPE = torch.float16
DEVICE = torch.device("cuda:0")

In [4]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
qs, qs.shape

(tensor([[[[0.0000, 0.1000, 0.2000, 0.3000],
           [0.3999, 0.5000, 0.6001, 0.7002],
           [0.7998, 0.8999, 1.0000, 1.0996],
           [1.2002, 1.2998, 1.4004, 1.5000],
           [1.5996, 1.7002, 1.7998, 1.9004],
           [2.0000, 2.0996, 2.1992, 2.3008],
           [2.4004, 2.5000, 2.5996, 2.6992],
           [2.8008, 2.9004, 3.0000, 3.0996]]]], device='cuda:0',
        dtype=torch.float16),
 torch.Size([1, 1, 8, 4]))

In [ ]:
rs = qs @ ks.transpose(-1, -2)
rs, rs.shape

In [5]:
# pytorch version
rs = qs @ ks.transpose(-1, -2) @ vs
rs, rs.shape

(tensor([[[[ 4.8008,  4.8008,  4.8008,  4.8008],
           [17.5938, 17.5938, 17.5938, 17.5938],
           [30.3906, 30.3906, 30.3906, 30.3906],
           [43.1875, 43.1875, 43.1875, 43.1875],
           [56.0000, 56.0000, 56.0000, 56.0000],
           [68.8125, 68.8125, 68.8125, 68.8125],
           [81.6250, 81.6250, 81.6250, 81.6250],
           [94.3750, 94.3750, 94.3750, 94.3750]]]], device='cuda:0',
        dtype=torch.float16),
 torch.Size([1, 1, 8, 4]))

In [6]:
# cuda kernel
rs = qkvkernel(mat_Q=qs, mat_K=ks, mat_V=vs)
rs, rs.shape

before kernel dispatch - float16!
B: 1, NH: 1, S: 8, DH: 4
blocksxy: 1-2, threads: 4-4
In Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4
In Kernel: qTileEnd: 1, kvTileEnd: 1
2-B<0,1>T<0,0> - qTile[4][0]: 1.599609
1-B<0,0>T<0,3> - qTile[3][0]: 1.200195
2-B<0,1>T<0,0> - kTile[0][0]: 0.000000
1-B<0,0>T<0,3> - kTile[0][0]: 1.000000
2-B<0,1>T<0,0> - qTile[4][1]: 1.700195
1-B<0,0>T<0,3> - qTile[3][1]: 1.299805
2-B<0,1>T<0,0> - kTile[0][1]: 0.000000
1-B<0,0>T<0,3> - kTile[0][1]: 1.000000
2-B<0,1>T<0,0> - qTile[4][2]: 1.799805
1-B<0,0>T<0,3> - qTile[3][2]: 1.400391
2-B<0,1>T<0,0> - kTile[0][2]: 0.000000
1-B<0,0>T<0,3> - kTile[0][2]: 1.000000
2-B<0,1>T<0,0> - qTile[4][3]: 1.900391
1-B<0,0>T<0,3> - qTile[3][3]: 1.500000
2-B<0,1>T<0,0> - kTile[0][3]: 0.000000
1-B<0,0>T<0,3> - kTile[0][3]: 1.000000


(tensor([[[[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]]]], device='cuda:0', dtype=torch.float16),
 torch.Size([1, 1, 8, 4]))

In [ ]:
qs

### Matmul kernel Test from vlstm_v2

In [ ]:
# from src.vlstm_v3.interface import testkernel, copykernel, mmkernelv1

In [7]:
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")

In [ ]:
S = 8 # sequence length
DH = 8 # hidden size

In [ ]:
matA = torch.arange((2*S * DH), device=DEVICE, dtype=DTYPE).reshape((2*S, DH))
matB = torch.ones((DH, S), device=DEVICE, dtype=DTYPE)
matA.shape, matB.shape, matA, matA.sum(-1)

In [ ]:
# pytorch
pt_out = matA @ matB
pt_out, pt_out.shape

In [ ]:
matA.is_contiguous(), matB.is_contiguous()

In [ ]:
cu_out = mmkernelv1(mat_A=matA, mat_B=matB)
cu_out, cu_out.shape

In [ ]:
matA[9]

In [ ]:
matA[9].cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.bfloat16, device=torch.device('cuda:0')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.bfloat16, device=torch.device('cpu')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.float32, device=torch.device('cpu')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.float16, device=torch.device('cuda:0')).cumsum(-1)

In [8]:
# cu_out = mmkernelv2(mat_A=matA, mat_B=matB)
# cu_out, cu_out.shape

In [ ]:
# mat @ mat.T @ mat

### pytorch version

In [ ]:
H = 6 # hidden size
S = 5 # seq len
B = 1 # batch size
NH = 2 # num heads
DH = H // NH # dim per head
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")
assert H % NH == 0

In [ ]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ds = torch.rand((B, NH, S, S), device=DEVICE, dtype=DTYPE)

max_log_D, _ = torch.max(ds.view(B, NH, -1), dim=-1, keepdim=True)  # (B, NH, 1)
log_D_matrix_stabilized = ds - max_log_D.unsqueeze(-1)  # (B, NH, S, S) = (B, NH, S, S) - (B, NH, 1, 1)
D_matrix = torch.exp(log_D_matrix_stabilized)  # (B, NH, S, S)
mval = torch.exp(-max_log_D.unsqueeze(-1))